# supplementary2

## Perform 2-way-anova

## Load Normalized data and metadata

In [ ]:
# Load necessary libraries
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

The following object is masked from 'package:purrr':

    some

Mismatches in metadata not in melted_normalized_data: 50 

Mismatches in melted_normalized_data not in metadata:  

7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].

# Find number of significant compounds for each effect

In [ ]:
significance_level <- 0.05
significant_climate <- anova_results_df %>%
  filter(Term == "Climate" & `Pr(>F)` < significance_level) %>%
  pull(GlobalID) %>%
  unique() %>%
  length()

significant_water <- anova_results_df %>%
  filter(Term == "Water" & `Pr(>F)` < significance_level) %>%
  pull(GlobalID) %>%
  unique() %>%
  length()

significant_interaction <- anova_results_df %>%
  filter(Term == "Climate:Water" & `Pr(>F)` < significance_level) %>%
  pull(GlobalID) %>%
  unique() %>%
  length()

# Print the results
cat("Number of significant compounds for Climate:", significant_climate, "\n")

Number of significant compounds for Climate: 206 

Number of significant compounds for Water: 563 

Number of significant compounds for Interaction: 148 

# Get list of all significant compounds

In [ ]:
significant_compounds <- anova_results_df %>%
  filter(`Pr(>F)` < significance_level) %>%
  pull(GlobalID) %>%
  unique()

# Create a data frame to store significance types for each compound
significance_types <- anova_results_df %>%
  filter(GlobalID %in% significant_compounds, `Pr(>F)` < significance_level) %>%
  group_by(GlobalID) %>%
  summarize(
    Climate = any(Term == "Climate" & `Pr(>F)` < significance_level),
    Water = any(Term == "Water" & `Pr(>F)` < significance_level),
    Interaction = any(Term == "Climate:Water" & `Pr(>F)` < significance_level)
  ) %>%
  mutate(
    SignificanceType = case_when(
      Climate & Water & Interaction ~ "All",
      Climate & Water ~ "Climate & Water",
      Climate & Interaction ~ "Climate & Interaction",
      Water & Interaction ~ "Water & Interaction",
      Climate ~ "Climate",
      Water ~ "Water",
      Interaction ~ "Interaction"
    )
  )

# Prepare data for heatmap

In [ ]:
library(dplyr)
library(tidyr)
library(tibble)
library(pheatmap)

# Assuming `merged_data`, `significant_compounds`, `metadata`, and `significance_types` are already loaded

# Prepare the heatmap data
heatmap_data <- merged_data %>%
  filter(GlobalID %in% significant_compounds) %>%
  group_by(GlobalID, Sample) %>%
  summarize(Value = mean(Value, na.rm = TRUE), .groups = 'drop') %>%
  pivot_wider(names_from = Sample, values_from = Value) %>%
  column_to_rownames("GlobalID")

# Check if there are still duplicate row names
if (any(duplicated(rownames(heatmap_data)))) {
  cat("Warning: There are still duplicate GlobalIDs. Adding a suffix to make them unique.\n")
  rownames(heatmap_data) <- make.unique(rownames(heatmap_data))
}

# Remove non-numeric columns
heatmap_data <- heatmap_data %>% select_if(is.numeric)

# Create annotation data frame
annotation_df <- metadata %>%
  dplyr::select(Sample, Climate = Type1, Water = Type2) %>%
  tibble::column_to_rownames("Sample")

# Ensure all samples in heatmap_data are present in annotation_df
common_samples <- intersect(colnames(heatmap_data), rownames(annotation_df))
heatmap_data <- heatmap_data[, common_samples]
annotation_df <- annotation_df[common_samples, ]

# Create annotation colors
ann_colors <- list(
  Climate = c("Ambient" = "#66c2a5", "Warmed" = "#fc8d62"),
  Water = c("Dry" = "#8da0cb", "Wet" = "#e78ac3")
)

# Assuming the first column in significance_types is the identifier for compounds
row_annotation <- significance_types %>%
  dplyr::select(1, SignificanceType) %>%
  tibble::column_to_rownames(var = names(.)[1])

# Ensure row_annotation matches heatmap_data
row_annotation <- row_annotation[rownames(heatmap_data), , drop = FALSE]

# Check if any NAs were introduced
if (any(is.na(row_annotation))) {
  cat("Warning: Some compounds in heatmap_data are missing from significance_types.\n")
  # Remove rows with NA if necessary
  row_annotation <- row_annotation[complete.cases(row_annotation), , drop = FALSE]
}

# Create annotation colors based on actual levels
ann_colors <- list(
  Climate = c("Ambient" = "#66c2a5", "Future" = "#fc8d62"),
  Water = c("No_drought" = "#8da0cb", "Drought" = "#e78ac3")
)

# Create significance type colors
sig_colors <- c(
  "Climate" = "#e41a1c",
  "Water" = "#377eb8",
  "Interaction" = "#4daf4a",
  "Climate & Water" = "#984ea3",
  "Climate & Interaction" = "#ff7f00",
  "Water & Interaction" = "#ffff33",
  "All" = "#a65628"
)

# Combine annotation colors
combined_ann_colors <- c(ann_colors, list(SignificanceType = sig_colors))

# Save the heatmap to a PDF file
pdf("significant_compounds_heatmap.pdf", width = 12, height = 8)
p <- pheatmap(
  heatmap_data,
  scale = "row",
  clustering_distance_rows = "correlation",
  clustering_distance_cols = "correlation",
  annotation_col = annotation_df,
  annotation_row = row_annotation,
  annotation_colors = combined_ann_colors,
  show_rownames = FALSE,
  show_colnames = FALSE,
  main = "Hierarchical Clustering Heatmap of Significant Compounds"
)
dev.off()

pdf 
  3 